In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20130101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

select * from stock_day where date>=20130101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(5390287, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5390287 entries, (000001.SZ, 2013-01-04 00:00:00) to (688399.SH, 2020-04-21 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 407.2 MB


open        high         low       close  \
code      date                                                         
688399.SH 2020-04-15   80.750000   84.339996   80.300003   82.000000   
          2020-04-16   83.400002   96.000000   82.440002   94.900002   
          2020-04-17   95.949997   95.949997   88.879997   92.000000   
          2020-04-20   95.790001  109.970001   95.199997  108.730003   
          2020-04-21  108.000000  118.199997  102.000000  114.440002   

                               vol       amt  adj_factor         avg  \
code      date                                                         
688399.SH 2020-04-15  10474.320312  0.863751         1.0   82.463715   
          2020-04-16  31260.830078  2.815096         1.0   90.051849   
          2020-04-17  24047.009766  2.208113         1.0   91.824860   
          2020-04-20  34636.898438  3.590274         1.0  103.654602   
          2020-04-21  33081.828125  3.580782         1.0  108.240166   

                          close0        avg0        pct  
code      date                                           
688399.SH 2020-04-15   82.000000   82.463715   2.641129  
          2020-04-16   94.900002   90.051849  15.731705  
          2020-04-17   92.000000   91.824860  -3.055853  
          2020-04-20  108.730003  103.654602  18.184780  
          2020-04-21  114.440002  108.240166   5.251539

In [4]:
# df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
# df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
# #
# df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
# df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

# df_d_basic = df_d_basic.astype('float32')
# print(df_d_basic.columns)
# print(df_d_basic.shape)
# df_d_basic.info(memory_usage='deep')
# df_d_basic.index.tail()

select * from stock_daily_basic where date>=20100101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(2913386, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2913386 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-17 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 189.0 MB


AttributeError: 'MultiIndex' object has no attribute 'tail'

In [5]:
df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
    df['{:d}ma_avg'.format(k)]=df.reset_index('code').groupby('code')['amt'].rolling(k).sum()\
            /df.reset_index('code').groupby('code')['vol'].rolling(k).sum()*1e6
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

for col2 in ["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '60ma', '60ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '60ma', '60ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg', 'close/5ma', 'close/10ma',
       'close/20ma', 'close/30ma', 'close/60ma', 'close/120ma', 'close/250ma',
       'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg',
       'close/60ma_avg', 'close/120ma_avg', 'close/250ma_avg'],
      dtype='object')


In [9]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r_spl = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()
df_r_spc = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

features = []
for col2 in ["{}ma".format(k) for k in periods]+["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
print(features)
# result_50 = ml.assess_feature3(df[features],df_r.r,q_bin=50)
result_100_spl = ml.assess_feature3(df[features],df_r_spl.r,q_bin=100)
result_100_spc = ml.assess_feature3(df[features],df_r_spc.r,q_bin=100)

# print(result_50.sort_values(['median_std','mean_std'],ascending=False))
result_100_spl.sort_values(['median_std','mean_std'],ascending=False)
result_100_spc.sort_values(['median_std','mean_std'],ascending=False)


['close/5ma', 'close/10ma', 'close/20ma', 'close/30ma', 'close/60ma', 'close/120ma', 'close/250ma', 'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg', 'close/60ma_avg', 'close/120ma_avg', 'close/250ma_avg']
close/10ma 
                       mean    median
bin                                 
(0.5808, 0.8483]  0.001142 -0.000526
(0.8483, 0.8813]  0.004385  0.001295
(0.8813, 0.8991]  0.004299 -0.000335
(0.8991, 0.9113]  0.003977 -0.001766
(0.9113, 0.9202]  0.004181 -0.002849
(0.9202, 0.9272]  0.004341 -0.003645
(0.9272, 0.933]   0.004384 -0.003863
(0.933, 0.9379]   0.004031 -0.005128
(0.9379, 0.9421]  0.004009 -0.005668
(0.9421, 0.9458]  0.004176 -0.006208
(0.9458, 0.9491]  0.003303 -0.007362
(0.9491, 0.9521]  0.003107 -0.007730
(0.9521, 0.9548]  0.003734 -0.007489
(0.9548, 0.9573]  0.003133 -0.007928
(0.9573, 0.9595]  0.003592 -0.007968
(0.9595, 0.9617]  0.002164 -0.009009
(0.9617, 0.9637]  0.002025 -0.008883
(0.9637, 0.9655]  0.002603 -0.009442
(0.9655, 0.9672]  0.

close/120ma 
                      mean    median
bin                                
(0.286, 0.684]   0.029159  0.017143
(0.684, 0.724]   0.017475  0.006977
(0.724, 0.748]   0.014259  0.004273
(0.748, 0.766]   0.011924  0.001869
(0.766, 0.78]    0.009818 -0.000415
(0.78, 0.792]    0.007251 -0.002382
(0.792, 0.803]   0.007100 -0.002845
(0.803, 0.812]   0.006389 -0.003623
(0.812, 0.82]    0.005671 -0.004079
(0.82, 0.828]    0.004644 -0.004926
(0.828, 0.835]   0.003903 -0.005972
(0.835, 0.842]   0.003592 -0.005967
(0.842, 0.848]   0.003870 -0.005882
(0.848, 0.853]   0.003370 -0.006637
(0.853, 0.859]   0.002893 -0.007547
(0.859, 0.864]   0.002699 -0.007218
(0.864, 0.869]   0.002045 -0.008043
(0.869, 0.873]   0.002495 -0.007209
(0.873, 0.878]   0.001940 -0.007962
(0.878, 0.882]   0.002338 -0.007692
(0.882, 0.887]   0.001862 -0.008333
(0.887, 0.891]   0.001304 -0.008830
(0.891, 0.895]   0.000920 -0.008835
(0.895, 0.898]   0.001438 -0.008772
(0.898, 0.902]   0.001074 -0.009038
(0.902, 0.906]

close/20ma_avg 
                     mean    median
bin                               
(0.418, 0.782]  0.012036  0.014583
(0.782, 0.824]  0.001070  0.000000
(0.824, 0.847]  0.003748 -0.001229
(0.847, 0.862]  0.003450 -0.003320
(0.862, 0.874]  0.002606 -0.004541
(0.874, 0.884]  0.002721 -0.005155
(0.884, 0.892]  0.002510 -0.006307
(0.892, 0.899]  0.002415 -0.007143
(0.899, 0.905]  0.002350 -0.007257
(0.905, 0.91]   0.002406 -0.007973
(0.91, 0.915]   0.002875 -0.007326
(0.915, 0.919]  0.002435 -0.008197
(0.919, 0.923]  0.002619 -0.008419
(0.923, 0.927]  0.002539 -0.008783
(0.927, 0.93]   0.002875 -0.008931
(0.93, 0.933]   0.002614 -0.008974
(0.933, 0.936]  0.002620 -0.009172
(0.936, 0.939]  0.002258 -0.009836
(0.939, 0.941]  0.001932 -0.010152
(0.941, 0.944]  0.002147 -0.009818
(0.944, 0.946]  0.001973 -0.010753
(0.946, 0.949]  0.001507 -0.010268
(0.949, 0.951]  0.002390 -0.009842
(0.951, 0.953]  0.001536 -0.010851
(0.953, 0.955]  0.001840 -0.010169
(0.955, 0.957]  0.001499 -0.010762
(0.

close/30ma 
                     mean    median
bin                               
(0.368, 0.767]  0.021221  0.023763
(0.767, 0.811]  0.006555  0.005634
(0.811, 0.835]  0.006263  0.002165
(0.835, 0.851]  0.005928  0.000000
(0.851, 0.864]  0.004895 -0.003115
(0.864, 0.874]  0.004545 -0.004525
(0.874, 0.883]  0.004518 -0.005015
(0.883, 0.89]   0.004429 -0.005629
(0.89, 0.897]   0.003887 -0.006309
(0.897, 0.902]  0.004669 -0.006667
(0.902, 0.908]  0.003676 -0.006667
(0.908, 0.912]  0.003133 -0.007708
(0.912, 0.917]  0.003309 -0.007792
(0.917, 0.921]  0.003784 -0.007734
(0.921, 0.925]  0.003970 -0.007859
(0.925, 0.928]  0.003451 -0.007692
(0.928, 0.931]  0.003171 -0.008000
(0.931, 0.935]  0.002296 -0.009148
(0.935, 0.938]  0.002355 -0.009302
(0.938, 0.94]   0.002799 -0.009009
(0.94, 0.943]   0.002550 -0.009494
(0.943, 0.946]  0.001972 -0.010000
(0.946, 0.948]  0.002032 -0.009589
(0.948, 0.951]  0.001833 -0.009816
(0.951, 0.953]  0.001159 -0.010443
(0.953, 0.955]  0.001134 -0.010563
(0.955,

close/5ma_avg 
                       mean    median
bin                                 
(0.6025, 0.8875]  0.005230  0.002069
(0.8875, 0.9111]  0.000671 -0.002272
(0.9111, 0.9239]  0.002093 -0.003898
(0.9239, 0.9329]  0.003940 -0.003236
(0.9329, 0.9396]  0.005070 -0.003341
(0.9396, 0.945]   0.005884 -0.003681
(0.945, 0.9493]   0.005353 -0.004098
(0.9493, 0.953]   0.005650 -0.004360
(0.953, 0.9561]   0.006070 -0.004392
(0.9561, 0.9588]  0.004859 -0.005660
(0.9588, 0.9613]  0.004976 -0.005556
(0.9613, 0.9635]  0.005006 -0.006173
(0.9635, 0.9655]  0.004706 -0.006477
(0.9655, 0.9673]  0.003725 -0.007700
(0.9673, 0.969]   0.003445 -0.007887
(0.969, 0.9706]   0.003919 -0.007874
(0.9706, 0.9721]  0.002808 -0.009091
(0.9721, 0.9734]  0.002564 -0.008876
(0.9734, 0.9747]  0.002328 -0.009358
(0.9747, 0.976]   0.001737 -0.009714
(0.976, 0.9772]   0.002023 -0.009333
(0.9772, 0.9783]  0.002084 -0.009732
(0.9783, 0.9793]  0.001659 -0.010135
(0.9793, 0.9804]  0.001312 -0.010498
(0.9804, 0.9814]  0.00

...

close/20ma_avg 
                     mean    median
bin                               
(0.418, 0.782]  0.012036  0.014583
(0.782, 0.824]  0.001070  0.000000
(0.824, 0.847]  0.003748 -0.001229
(0.847, 0.862]  0.003450 -0.003320
(0.862, 0.874]  0.002606 -0.004541
(0.874, 0.884]  0.002721 -0.005155
(0.884, 0.892]  0.002510 -0.006307
(0.892, 0.899]  0.002415 -0.007143
(0.899, 0.905]  0.002350 -0.007257
(0.905, 0.91]   0.002406 -0.007973
(0.91, 0.915]   0.002875 -0.007326
(0.915, 0.919]  0.002435 -0.008197
(0.919, 0.923]  0.002619 -0.008419
(0.923, 0.927]  0.002539 -0.008783
(0.927, 0.93]   0.002875 -0.008931
(0.93, 0.933]   0.002614 -0.008974
(0.933, 0.936]  0.002620 -0.009172
(0.936, 0.939]  0.002258 -0.009836
(0.939, 0.941]  0.001932 -0.010152
(0.941, 0.944]  0.002147 -0.009818
(0.944, 0.946]  0.001973 -0.010753
(0.946, 0.949]  0.001507 -0.010268
(0.949, 0.951]  0.002390 -0.009842
(0.951, 0.953]  0.001536 -0.010851
(0.953, 0.955]  0.001840 -0.010169
(0.955, 0.957]  0.001499 -0.010762
(0.

close/30ma 
                     mean    median
bin                               
(0.368, 0.767]  0.021221  0.023763
(0.767, 0.811]  0.006555  0.005634
(0.811, 0.835]  0.006263  0.002165
(0.835, 0.851]  0.005928  0.000000
(0.851, 0.864]  0.004895 -0.003115
(0.864, 0.874]  0.004545 -0.004525
(0.874, 0.883]  0.004518 -0.005015
(0.883, 0.89]   0.004429 -0.005629
(0.89, 0.897]   0.003887 -0.006309
(0.897, 0.902]  0.004669 -0.006667
(0.902, 0.908]  0.003676 -0.006667
(0.908, 0.912]  0.003133 -0.007708
(0.912, 0.917]  0.003309 -0.007792
(0.917, 0.921]  0.003784 -0.007734
(0.921, 0.925]  0.003970 -0.007859
(0.925, 0.928]  0.003451 -0.007692
(0.928, 0.931]  0.003171 -0.008000
(0.931, 0.935]  0.002296 -0.009148
(0.935, 0.938]  0.002355 -0.009302
(0.938, 0.94]   0.002799 -0.009009
(0.94, 0.943]   0.002550 -0.009494
(0.943, 0.946]  0.001972 -0.010000
(0.946, 0.948]  0.002032 -0.009589
(0.948, 0.951]  0.001833 -0.009816
(0.951, 0.953]  0.001159 -0.010443
(0.953, 0.955]  0.001134 -0.010563
(0.955,

close/5ma_avg 
                       mean    median
bin                                 
(0.6025, 0.8875]  0.005230  0.002069
(0.8875, 0.9111]  0.000671 -0.002272
(0.9111, 0.9239]  0.002093 -0.003898
(0.9239, 0.9329]  0.003940 -0.003236
(0.9329, 0.9396]  0.005070 -0.003341
(0.9396, 0.945]   0.005884 -0.003681
(0.945, 0.9493]   0.005353 -0.004098
(0.9493, 0.953]   0.005650 -0.004360
(0.953, 0.9561]   0.006070 -0.004392
(0.9561, 0.9588]  0.004859 -0.005660
(0.9588, 0.9613]  0.004976 -0.005556
(0.9613, 0.9635]  0.005006 -0.006173
(0.9635, 0.9655]  0.004706 -0.006477
(0.9655, 0.9673]  0.003725 -0.007700
(0.9673, 0.969]   0.003445 -0.007887
(0.969, 0.9706]   0.003919 -0.007874
(0.9706, 0.9721]  0.002808 -0.009091
(0.9721, 0.9734]  0.002564 -0.008876
(0.9734, 0.9747]  0.002328 -0.009358
(0.9747, 0.976]   0.001737 -0.009714
(0.976, 0.9772]   0.002023 -0.009333
(0.9772, 0.9783]  0.002084 -0.009732
(0.9783, 0.9793]  0.001659 -0.010135
(0.9793, 0.9804]  0.001312 -0.010498
(0.9804, 0.9814]  0.00

,mean_std,median_std,mean_q96%,median_q96%,mean_q4%,median_q4%
close/60ma,0.003340,0.004305,0.006516,-0.002918,0.000471,-0.011036
close/60ma_avg,0.003392,0.004275,0.006021,-0.002554,0.000521,-0.010731
close/30ma,0.002415,0.004108,0.004936,-0.002991,0.000594,-0.010567
close/120ma_avg,0.004044,0.004064,0.009205,0.000029,0.000072,-0.010668
close/30ma_avg,0.002439,0.004059,0.004749,-0.004023,0.000476,-0.010759
close/120ma,0.003863,0.003930,0.009902,-0.000324,-0.000141,-0.010794
close/20ma,0.001762,0.003255,0.004477,-0.003547,0.000882,-0.010989
close/20ma_avg,0.001744,0.003072,0.004537,-0.004492,0.000816,-0.010859
close/10ma,0.001961,0.002350,0.004799,-0.002806,0.000541,-0.011052
close/5ma,0.007807,0.002149,0.005644,-0.003661,0.000582,-0.010503


,mean_std,median_std,mean_q96%,median_q96%,mean_q4%,median_q4%
close/60ma,0.003340,0.004305,0.006516,-0.002918,0.000471,-0.011036
close/60ma_avg,0.003392,0.004275,0.006021,-0.002554,0.000521,-0.010731
close/30ma,0.002415,0.004108,0.004936,-0.002991,0.000594,-0.010567
close/120ma_avg,0.004044,0.004064,0.009205,0.000029,0.000072,-0.010668
close/30ma_avg,0.002439,0.004059,0.004749,-0.004023,0.000476,-0.010759
close/120ma,0.003863,0.003930,0.009902,-0.000324,-0.000141,-0.010794
close/20ma,0.001762,0.003255,0.004477,-0.003547,0.000882,-0.010989
close/20ma_avg,0.001744,0.003072,0.004537,-0.004492,0.000816,-0.010859
close/10ma,0.001961,0.002350,0.004799,-0.002806,0.000541,-0.011052
close/5ma,0.007807,0.002149,0.005644,-0.003661,0.000582,-0.010503


由上可知，avg类变量在等频分箱下差异更小，区分度弱于非avg，但在末端(1%)下的收益率明显高于非avg。